In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.chdir(r"C:\Users\cornuch\Desktop\SISE\Text_mining\projet_disney")    

In [3]:
df=pd.read_csv('Scrapping_Newport_Bay_Club.csv',sep=';')

In [4]:
df.head()

,Names,Country,room_type,nuitee,reservation_date,traveler_infos,date_review,review_title,grade_review,positive_review,negative_review,usefulness_review,hotel
0,Ludovic,La Réunion,Chambre Supérieure - Côté Lac,2,Octobre 2022,Famille,Commentaire envoyé le 9 octobre 2022,Inoubliable,10,Le petit déjeuner et l'excellent buffet pour l...,NaN,0,Disney's Newport Bay Club
1,Julien,France,Chambre Supérieure,1,Décembre 2022,Couple,Commentaire envoyé le 15 décembre 2022,Très bien,"8,0","Les navettes pour accéder au parc,\r\nLa taill...",R.a.s,0,Disney's Newport Bay Club
2,Celine,France,Chambre Supérieure,2,Décembre 2022,Famille,Commentaire envoyé le 14 décembre 2022,Agréable,"6,0",Û,NaN,0,Disney's Newport Bay Club
3,Diego,France,Chambre Supérieure,1,Novembre 2022,Famille,Commentaire envoyé le 30 novembre 2022,Agréable,"6,0",la proximité du parc Disneyland Paris,"la qualité de la chambre n’était pas top, vieu...",0,Disney's Newport Bay Club
4,Fabienne,Belgique,Chambre Supérieure,1,Novembre 2022,Famille,Commentaire envoyé le 18 novembre 2022,déçue qualité prix,"6,0",l emplacement,la propreté,0,Disney's Newport Bay Club


In [5]:
df.shape

(2276, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2276 entries, 0 to 2275
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Names              2276 non-null   object
 1   Country            2274 non-null   object
 2   room_type          2130 non-null   object
 3   nuitee             2276 non-null   int64 
 4   reservation_date   2276 non-null   object
 5   traveler_infos     2276 non-null   object
 6   date_review        2276 non-null   object
 7   review_title       2274 non-null   object
 8   grade_review       2276 non-null   object
 9   positive_review    627 non-null    object
 10  negative_review    494 non-null    object
 11  usefulness_review  2276 non-null   int64 
 12  hotel              2276 non-null   object
dtypes: int64(2), object(11)
memory usage: 231.3+ KB


## Ajout dates

In [7]:
def clean_date_ajout(df):
    df_moisreview=df['date_review'].map(str)
    for i in range(df.shape[0]):
        df_moisreview[i]=df_moisreview[i].split()[4]
    df_moisreservation=df['reservation_date'].map(str)
    for i in range(df.shape[0]):
        df_moisreservation[i]=df_moisreservation[i].split()[0].lower()
        df_anneereservation=df['reservation_date'].map(str)
    for i in range(df.shape[0]):
        df_anneereservation[i]=df_anneereservation[i].split()[1]
        from time import strptime
    import locale
    locale.setlocale(locale.LC_TIME,'')

    list_mois_num = [strptime(moisreservation,'%B').tm_mon for moisreservation in df_moisreservation]
    list_mois_com = [strptime(moisreview,'%B').tm_mon for moisreview in df_moisreview]
    delai = [com - num if com-num>-1 else 12+(com-num) for com, num in zip(list_mois_com, list_mois_num)]

    d = {'month_str': df_moisreservation, 'month_num': list_mois_num, 'year' : df_anneereservation,  'delay_comment': delai}
    df_date = pd.DataFrame(data=d)
    df = pd.concat([df,df_date], join = 'outer', axis = 1)
    return(df)

In [8]:
def ajout_levels(df):
    conditionlist_note = [
    (df['grade_review'] >= 8) ,
    (df['grade_review'] > 5) & (df['grade_review'] <8),
    (df['grade_review'] <= 5)]
    choicelist_note = [2,1,0]
    df['level_grade_review'] = np.select(conditionlist_note, choicelist_note, default='Not Specified')
    conditionlist_hotel = [
    (df['hotel'] == "Disney's Newport Bay Club")]
    choicelist_hotel = [6]
    df['level_hotel'] = np.select(conditionlist_hotel, choicelist_hotel, default='Not Specified')
    return(df)

In [9]:
def recodage(df):
    # Liste des colonnes à transtyper en numeric
    var_tofloat = ['grade_review']
    for i in var_tofloat:
        df[i] = df[i].str.replace(",",".")
        df[i] = pd.to_numeric(df[i], downcast="float")
    return(df)

In [10]:
df = recodage(df)

In [11]:
df = ajout_levels(df)

In [12]:
df = clean_date_ajout(df)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2276 entries, 0 to 2275
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Names               2276 non-null   object 
 1   Country             2274 non-null   object 
 2   room_type           2130 non-null   object 
 3   nuitee              2276 non-null   int64  
 4   reservation_date    2276 non-null   object 
 5   traveler_infos      2276 non-null   object 
 6   date_review         2276 non-null   object 
 7   review_title        2274 non-null   object 
 8   grade_review        2276 non-null   float32
 9   positive_review     627 non-null    object 
 10  negative_review     494 non-null    object 
 11  usefulness_review   2276 non-null   int64  
 12  hotel               2276 non-null   object 
 13  level_grade_review  2276 non-null   object 
 14  level_hotel         2276 non-null   object 
 15  month_str           2276 non-null   object 
 16  month_

In [14]:
df = df.drop_duplicates(keep='first')

In [15]:
df.drop(df[(df.delay_comment >3)].index , inplace=True)

In [16]:
df=df.reset_index()

In [17]:
df_clean_newport=df.copy()

In [18]:
df_clean_newport.to_csv("C:/Users/cornuch/Desktop/SISE/Text_Mining/projet_disney/df_clean_newport.csv",sep=';', index=False)